In [2]:
import requests
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
import plotly.graph_objects as go

# Função para coletar dados da API
def fetch_brent_data(api_key, start_date, end_date):
    url = "https://api.eia.gov/v2/petroleum/pri/spt/data/"
    params = {
        'api_key': api_key,
        'facets[product][]': 'EPCBRENT',
        'data[]': 'value',
        'frequency': 'daily',
        'start': start_date,
        'end': end_date,
        'offset': 0
    }
    all_data = []
    while True:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            if 'response' in data and 'data' in data['response']:
                brent_data = data['response']['data']
                all_data.extend(brent_data)
                if len(brent_data) < 5000:
                    break
                params['offset'] += 5000
            else:
                raise ValueError("Erro: Nenhum dado encontrado na resposta.")
        else:
            raise ConnectionError(f"Erro na requisição: Status Code {response.status_code}")
    df = pd.DataFrame(all_data)
    df = df[['period', 'value']].rename(columns={'period': 'ds', 'value': 'y'})
    df['ds'] = pd.to_datetime(df['ds'])
    df['y'] = pd.to_numeric(df['y'], errors='coerce')
    return df.dropna()

# Função para calcular métricas
def calculate_metrics(y_true, y_pred):
    non_zero_mask = y_true != 0
    y_true = y_true[non_zero_mask]
    y_pred = y_pred[non_zero_mask]
    
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    accuracy = 100 - mape
    return {"MAE": mae, "MSE": mse, "RMSE": rmse, "MAPE": mape, "Accuracy %": accuracy}

# Função para validação cruzada
def validate_model(model, horizon="30 days"):
    df_cv = cross_validation(model, initial='730 days', period='180 days', horizon=horizon)
    df_performance = performance_metrics(df_cv)
    return df_performance

# Função para treinar e prever usando Prophet
def train_and_forecast(data, prediction_days):
    train = data.iloc[:-prediction_days]
    test = data.iloc[-prediction_days:]
    model = Prophet()
    model.fit(train)
    future = model.make_future_dataframe(periods=prediction_days)
    forecast = model.predict(future)
    forecast_filtered = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(prediction_days)
    return model, forecast, forecast_filtered, test

# Função para visualização interativa
def interactive_plot(data, forecast, prediction_days):
    # Garantir que somente as colunas necessárias são usadas
    data = data[['ds', 'y']].dropna()

    fig = go.Figure()

    # Adicionar os dados históricos
    fig.add_trace(go.Scatter(
        x=data['ds'], 
        y=data['y'], 
        mode='lines', 
        name='Histórico',
        line=dict(color='blue', dash='solid')  # Linha sólida para histórico
    ))

    # Adicionar previsão
    fig.add_trace(go.Scatter(
        x=forecast['ds'], 
        y=forecast['yhat'], 
        mode='lines', 
        name='Previsão', 
        line=dict(dash='dot', color='orange')  # Linha pontilhada para previsão
    ))

    # Adicionar limites de confiança
    fig.add_trace(go.Scatter(
        x=forecast['ds'], 
        y=forecast['yhat_upper'], 
        mode='lines', 
        name='Limite Superior', 
        line=dict(color='rgba(204, 204, 204, 0.6)'),
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=forecast['ds'], 
        y=forecast['yhat_lower'], 
        mode='lines', 
        name='Limite Inferior', 
        line=dict(color='rgba(204, 204, 204, 0.6)'),
        fill='tonexty',
        fillcolor='rgba(204, 204, 204, 0.3)',
        showlegend=True
    ))

    # Linha de início da previsão
    start_date = data['ds'].iloc[-prediction_days]
    fig.add_vline(x=start_date, line=dict(color="red", dash="dash"), name="Início da Previsão")

    fig.update_layout(
        title="Previsão do Preço do Petróleo Brent",
        xaxis_title="Data",
        yaxis_title="Preço Brent (USD)",
        legend_title="Legenda",
        template="plotly_white"
    )
    fig.show()

# Pipeline principal
def main():
    # Configuração inicial
    api_key = "kjZJakW4UuR2fvPQPEoC5c1Ngyvfj96lnYUj9rcJ"
    start_date = "1987-05-25"
    end_date = "2024-11-30"
    prediction_days = 30  # Altere para 30, 60 ou 90 conforme necessário
    
    print("Coletando dados da API...")
    raw_data = fetch_brent_data(api_key, start_date, end_date)
    
    print("Total de registros coletados:", len(raw_data))

    print("Treinando o modelo e gerando previsões...")
    model, forecast, forecast_filtered, test = train_and_forecast(raw_data, prediction_days)

    print("Calculando métricas de teste...")
    metrics = calculate_metrics(test['y'].values, forecast_filtered['yhat'].values)
    print("Métricas do Modelo:")
    for key, value in metrics.items():
        print(f"{key}: {value:.2f}")
    
    print("Realizando validação cruzada...")
    performance = validate_model(model, horizon=f"{prediction_days} days")
    print("Métricas da Validação Cruzada:")
    print(performance)

    print("Plotando os resultados...")
    interactive_plot(raw_data, forecast, prediction_days)

# Execução
if __name__ == "__main__":
    main()


c:\Users\Abdala\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Coletando dados da API...
Total de registros coletados: 9519
Treinando o modelo e gerando previsões...


12:21:58 - cmdstanpy - INFO - Chain [1] start processing
12:22:04 - cmdstanpy - INFO - Chain [1] done processing


Calculando métricas de teste...
Métricas do Modelo:
MAE: 8.84
MSE: 101.88
RMSE: 10.09
MAPE: 11.48
Accuracy %: 88.52
Realizando validação cruzada...


  0%|          | 0/72 [00:00<?, ?it/s]12:22:07 - cmdstanpy - INFO - Chain [1] start processing
12:22:08 - cmdstanpy - INFO - Chain [1] done processing
  1%|▏         | 1/72 [00:00<00:36,  1.95it/s]12:22:08 - cmdstanpy - INFO - Chain [1] start processing
12:22:08 - cmdstanpy - INFO - Chain [1] done processing
  3%|▎         | 2/72 [00:01<00:37,  1.89it/s]12:22:08 - cmdstanpy - INFO - Chain [1] start processing
12:22:09 - cmdstanpy - INFO - Chain [1] done processing
  4%|▍         | 3/72 [00:01<00:36,  1.88it/s]12:22:09 - cmdstanpy - INFO - Chain [1] start processing
12:22:09 - cmdstanpy - INFO - Chain [1] done processing
  6%|▌         | 4/72 [00:02<00:39,  1.74it/s]12:22:10 - cmdstanpy - INFO - Chain [1] start processing
12:22:10 - cmdstanpy - INFO - Chain [1] done processing
  7%|▋         | 5/72 [00:02<00:41,  1.62it/s]12:22:10 - cmdstanpy - INFO - Chain [1] start processing
12:22:11 - cmdstanpy - INFO - Chain [1] done processing
  8%|▊         | 6/72 [00:04<00:54,  1.21it/s]12:22:12

Métricas da Validação Cruzada:
   horizon         mse       rmse        mae      mape     mdape     smape  \
0   3 days  283.381502  16.833939  11.282460  0.228705  0.160161  0.216242   
1   4 days  289.684957  17.020134  11.362401  0.231459  0.160161  0.217192   
2   5 days  307.551594  17.537149  11.772368  0.235813  0.170909  0.222105   
3   6 days  308.417427  17.561817  11.983050  0.230646  0.186252  0.223462   
4   7 days  272.940574  16.520913  11.448543  0.216890  0.184942  0.216768   
5   8 days  278.354133  16.683948  11.468008  0.216790  0.176522  0.215445   
6   9 days  272.767419  16.515672  11.220980  0.221438  0.141925  0.215362   
7  10 days  295.784298  17.198381  11.602519  0.231872  0.156004  0.218767   
8  11 days  285.020059  16.882537  11.325036  0.225910  0.160430  0.213220   
9  12 days  302.978162  17.406268  11.668210  0.229243  0.170753  0.216412   
10 13 days  315.207760  17.754091  11.901235  0.229503  0.180710  0.221638   
11 14 days  280.553206  16.749723

c:\Users\Abdala\AppData\Local\Programs\Python\Python312\Lib\site-packages\_plotly_utils\basevalidators.py:107: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



ModuleNotFoundError: No module named 'distutils'